# Postgres Methods & Experimentation

In [1]:
import sys
import json
# import redis

# from qdrant_client import QdrantClient
from pathlib import Path
from typing import Optional, Sequence

# Add parent directory to path to import from implementation package
# Notebooks are in implementation/notebooks/, so we go up two levels to project root
sys.path.insert(0, str(Path().resolve().parent.parent))

from implementation.classes.movie import BaseMovie
from implementation.classes.enums import EntityCategory, Genre
from implementation.classes.schemas import MetadataFilters, ExtractedEntitiesResponse, LexicalCandidate, ExtractedEntityData
from db.ingest_movie import ingest_movie
from implementation.misc.helpers import tokenize_title_phrase
from db.postgres import pool, refresh_title_token_doc_frequency, fetch_movie_cards
from db.lexical_search import search_title_postings, lexical_search, _resolve_all_title_tokens

# # Redis
# r = redis.Redis(host="localhost", port=6379, decode_responses=True)

# # Qdrant
# qdrant = QdrantClient(host="localhost", port=6333)

## SETUP

In [2]:
# Open the pool and establish initial connections
await pool.open()
# Validate that connections actually work (fast-fail if Postgres is unreachable)
await pool.check()

In [3]:
# # Gracefully close all connections on shutdown
# await pool.close()

In [4]:
# LOAD MOVIES

json_path = Path("../../saved_imdb_movies.json")
with open(json_path, "r", encoding="utf-8") as f:
    movies_data = json.load(f)

# Convert each dictionary to an IMDBMovie object
movies = [BaseMovie(**movie_dict) for movie_dict in movies_data]

## INGESTION

In [5]:
# from tqdm.asyncio import tqdm

# # Run ingest_movie on every movie in parallel; tqdm.gather shows progress without blocking async
# await tqdm.gather(*[ingest_movie(movie) for movie in movies], desc="Ingesting movies")

In [6]:
# await refresh_title_token_doc_frequency()

## SEARCHING

In [7]:
E = ExtractedEntityData          # shorthand
Ent = ExtractedEntitiesResponse  # shorthand
F = MetadataFilters              # shorthand

LEXICAL_TEST_CASES: list[tuple[str, ExtractedEntitiesResponse, MetadataFilters, str]] = [

    # ── 1 ──────────────────────────────────────────────────────────────────
    (
        "Nolan sci-fi without Tom Hardy, post-2010",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Nolan",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Christopher Nolan"),
            E(candidate_entity_phrase="Tom Hardy",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=True,
              corrected_and_normalized_entity="Tom Hardy"),
        ]),
        F(min_release_ts=1_262_304_000, genres=[Genre.SCI_FI]),
        "HITS: Interstellar (Nolan, 2014, Sci-Fi, no Hardy). "
        "EXCLUDED: Inception (Hardy stars in it), Dark Knight (not Sci-Fi).",
    ),

    # ── 2 ──────────────────────────────────────────────────────────────────
    (
        "Spider-Man franchise + Miles Morales, animated PG",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Spider-Man",
              most_likely_category=EntityCategory.FRANCHISE,
              exclude_from_results=False,
              corrected_and_normalized_entity="Spider-Man"),
            E(candidate_entity_phrase="Miles Morales",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=False,
              corrected_and_normalized_entity="Miles Morales"),
        ]),
        F(min_maturity_rank=2, max_maturity_rank=2, genres=[Genre.ANIMATION]),
        "HITS: Spider-Man: Across the Spider-Verse "
        "(title tokens 'spider-man', character match, PG, Animation).",
    ),

    # ── 3 ──────────────────────────────────────────────────────────────────
    (
        "Spielberg + Harrison Ford adventures, exclude Star Wars",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Harrison Ford",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Harrison Ford"),
            E(candidate_entity_phrase="Spielberg",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Steven Spielberg"),
            E(candidate_entity_phrase="Star Wars",
              most_likely_category=EntityCategory.FRANCHISE,
              exclude_from_results=True,
              corrected_and_normalized_entity="Star Wars"),
        ]),
        F(genres=[Genre.ADVENTURE]),
        "HITS: Raiders of the Lost Ark (Ford + Spielberg, Adventure). "
        "EXCLUDED: Star Wars (franchise exclude via title tokens).",
    ),

    # ── 4 ──────────────────────────────────────────────────────────────────
    (
        "Peele + Aster R-rated horror, exclude James Wan",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Jordan Peele",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Jordan Peele"),
            E(candidate_entity_phrase="Ari Aster",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Ari Aster"),
            E(candidate_entity_phrase="James Wan",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=True,
              corrected_and_normalized_entity="James Wan"),
        ]),
        F(min_maturity_rank=4, max_maturity_rank=4, genres=[Genre.HORROR]),
        "HITS: Get Out (Peele, R, Horror), Hereditary (Aster, R, Horror). "
        "EXCLUDED: Saw (Wan directed), Insidious (Wan + PG-13).",
    ),

    # ── 5 ──────────────────────────────────────────────────────────────────
    (
        "David Fincher + Brad Pitt crime thrillers, 1990s",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="David Fincher",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="David Fincher"),
            E(candidate_entity_phrase="Brad Pitt",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Brad Pitt"),
        ]),
        F(min_release_ts=631_152_000, max_release_ts=946_598_400, genres=[Genre.CRIME, Genre.THRILLER]),
        "HITS: Se7en (Fincher + Pitt, Crime, 1995), "
        "Fight Club (Fincher + Pitt, Crime, 1999). Both match 2 entities.",
    ),

    # ── 6 ──────────────────────────────────────────────────────────────────
    (
        "Joker + Alfred characters at Warner Bros, PG-13",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Joker",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=False,
              corrected_and_normalized_entity="Joker"),
            E(candidate_entity_phrase="Alfred",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=False,
              corrected_and_normalized_entity="Alfred"),
            E(candidate_entity_phrase="Warner Bros",
              most_likely_category=EntityCategory.STUDIO,
              exclude_from_results=False,
              corrected_and_normalized_entity="Warner Bros."),
        ]),
        F(min_maturity_rank=3, max_maturity_rank=3),
        "HITS: The Dark Knight (Joker + Alfred + Warner Bros., PG-13). "
        "Matches 3 entities. No other film has both characters + studio.",
    ),

    # ── 7 ──────────────────────────────────────────────────────────────────
    (
        "John Williams + Lucasfilm, exclude Mark Hamill",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="John Williams",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="John Williams"),
            E(candidate_entity_phrase="Lucasfilm",
              most_likely_category=EntityCategory.STUDIO,
              exclude_from_results=False,
              corrected_and_normalized_entity="Lucasfilm"),
            E(candidate_entity_phrase="Mark Hamill",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=True,
              corrected_and_normalized_entity="Mark Hamill"),
        ]),
        F(),
        "HITS: Raiders of the Lost Ark (Williams + Lucasfilm, no Hamill). "
        "EXCLUDED: Star Wars (Hamill stars).",
    ),

    # ── 8 ──────────────────────────────────────────────────────────────────
    (
        "Denis Villeneuve + Amy Adams, exclude Blade Runner",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Villeneuve",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Denis Villeneuve"),
            E(candidate_entity_phrase="Amy Adams",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Amy Adams"),
            E(candidate_entity_phrase="Blade Runner",
              most_likely_category=EntityCategory.FRANCHISE,
              exclude_from_results=True,
              corrected_and_normalized_entity="Blade Runner"),
        ]),
        F(),
        "HITS: Arrival (Villeneuve + Adams). "
        "EXCLUDED: Blade Runner 2049 (franchise exclude via title tokens). "
        "Leap Year may surface (Adams, 1 entity) but ranks lower.",
    ),

    # ── 9 ──────────────────────────────────────────────────────────────────
    (
        "Michael Giacchino family animations under 110 min",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Michael Giacchino",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Michael Giacchino"),
        ]),
        F(genres=[Genre.ANIMATION, Genre.FAMILY], max_runtime=109),
        "HITS: Up (96m), Coco (105m), Zootopia (109m). "
        "All Giacchino + Animation + Family + under 110m.",
    ),

    # ── 10 ─────────────────────────────────────────────────────────────────
    (
        "Christian Bale, exclude Bruce Wayne, R-rated",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Christian Bale",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Christian Bale"),
            E(candidate_entity_phrase="Bruce Wayne",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=True,
              corrected_and_normalized_entity="Bruce Wayne"),
        ]),
        F(min_maturity_rank=4, max_maturity_rank=4),
        "HITS: American Psycho (Bale, R, no Bruce Wayne). "
        "EXCLUDED: The Dark Knight (Bruce Wayne + PG-13).",
    ),

    # ── 11 ─────────────────────────────────────────────────────────────────
    (
        "Nolan films, exclude Dark Knight, PG-13 over 140 min",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Nolan",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Christopher Nolan"),
            E(candidate_entity_phrase="Dark Knight",
              most_likely_category=EntityCategory.MOVIE_TITLE,
              exclude_from_results=True,
              corrected_and_normalized_entity="Dark Knight"),
        ]),
        F(min_maturity_rank=3, max_maturity_rank=3, min_runtime=141),
        "HITS: Inception (148m), Interstellar (169m). "
        "EXCLUDED: The Dark Knight (title exclude via tokens).",
    ),

    # ── 12 ─────────────────────────────────────────────────────────────────
    (
        "Elsa + Olaf at Disney Animation, animated musicals",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Elsa",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=False,
              corrected_and_normalized_entity="Elsa"),
            E(candidate_entity_phrase="Olaf",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=False,
              corrected_and_normalized_entity="Olaf"),
            E(candidate_entity_phrase="Disney Animation",
              most_likely_category=EntityCategory.STUDIO,
              exclude_from_results=False,
              corrected_and_normalized_entity="Walt Disney Animation Studios"),
        ]),
        F(genres=[Genre.ANIMATION, Genre.MUSICAL]),
        "HITS: Frozen (Elsa + Olaf + Disney Animation, Animation + Musical). "
        "Matches 3 entities. Adjust genre ID if Musical differs from Music (10402).",
    ),

    # ── 13 ─────────────────────────────────────────────────────────────────
    (
        "Alan Silvestri + Captain America franchise, PG-13",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Alan Silvestri",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Alan Silvestri"),
            E(candidate_entity_phrase="Captain America",
              most_likely_category=EntityCategory.FRANCHISE,
              exclude_from_results=False,
              corrected_and_normalized_entity="Captain America"),
        ]),
        F(min_maturity_rank=3, max_maturity_rank=3),
        "HITS: Captain America: The First Avenger (Silvestri + title match, PG-13), "
        "Avengers: Endgame (Silvestri, PG-13), Forrest Gump (Silvestri, PG-13). "
        "Cap: TFA scores highest (person + title). Others match Silvestri alone.",
    ),

    # ── 14 ─────────────────────────────────────────────────────────────────
    (
        "Cary Elwes movies, comedy or horror, under 105 min",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Cary Elwes",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Cary Elwes"),
        ]),
        F(genres=[Genre.COMEDY, Genre.HORROR], max_runtime=104),
        "HITS: The Princess Bride (Elwes, Comedy, 99m), "
        "Saw (Elwes, Horror, 103m).",
    ),

    # ── 15 ─────────────────────────────────────────────────────────────────
    (
        "Paramount comedies, PG-13, 1980s",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Paramount",
              most_likely_category=EntityCategory.STUDIO,
              exclude_from_results=False,
              corrected_and_normalized_entity="Paramount Pictures"),
        ]),
        F(min_release_ts=315_532_800, max_release_ts=631_065_600,
          min_maturity_rank=3, max_maturity_rank=3, genres=[Genre.COMEDY]),
        "HITS: Ferris Bueller's Day Off (Paramount, Comedy, PG-13, 1986), "
        "The Naked Gun (Paramount, Comedy, PG-13, 1988).",
    ),

    # ── 16 ─────────────────────────────────────────────────────────────────
    (
        "Jack Torrance + Patrick Bateman, R-rated dramas over 100 min",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Jack Torrance",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=False,
              corrected_and_normalized_entity="Jack Torrance"),
            E(candidate_entity_phrase="Patrick Bateman",
              most_likely_category=EntityCategory.CHARACTER,
              exclude_from_results=False,
              corrected_and_normalized_entity="Patrick Bateman"),
        ]),
        F(min_maturity_rank=4, genres=[Genre.DRAMA], min_runtime=101),
        "HITS: The Shining (Jack Torrance, R, Drama, 144m), "
        "American Psycho (Patrick Bateman, R, Drama, 102m).",
    ),

    # ── 17 ─────────────────────────────────────────────────────────────────
    (
        "Forrest Gump + Titanic titles, exclude DiCaprio",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Forrest Gump",
              most_likely_category=EntityCategory.MOVIE_TITLE,
              exclude_from_results=False,
              corrected_and_normalized_entity="Forrest Gump"),
            E(candidate_entity_phrase="Titanic",
              most_likely_category=EntityCategory.MOVIE_TITLE,
              exclude_from_results=False,
              corrected_and_normalized_entity="Titanic"),
            E(candidate_entity_phrase="DiCaprio",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=True,
              corrected_and_normalized_entity="Leonardo DiCaprio"),
        ]),
        None,
        "HITS: Forrest Gump (title match, Drama, no DiCaprio). "
        "EXCLUDED: Titanic (DiCaprio stars in it).",
    ),

    # ── 18 ─────────────────────────────────────────────────────────────────
    (
        "Ryan Gosling dramas, exclude A24, after 2000",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Ryan Gosling",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Ryan Gosling"),
            E(candidate_entity_phrase="A24",
              most_likely_category=EntityCategory.STUDIO,
              exclude_from_results=True,
              corrected_and_normalized_entity="A24"),
        ]),
        F(min_release_ts=946_684_800, genres=[Genre.DRAMA]),
        "HITS: Blade Runner 2049 (Gosling, Drama, Alcon/Columbia), "
        "The Notebook (Gosling, Drama, New Line). "
        "EXCLUDED: Past Lives (A24 studio, but Gosling isn't in it anyway).",
    ),

    # ── 19 ─────────────────────────────────────────────────────────────────
    (
        "Warner Bros sci-fi with Hardy + Reeves, R-rated",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Warner Bros",
              most_likely_category=EntityCategory.STUDIO,
              exclude_from_results=False,
              corrected_and_normalized_entity="Warner Bros."),
            E(candidate_entity_phrase="Tom Hardy",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Tom Hardy"),
            E(candidate_entity_phrase="Keanu Reeves",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=False,
              corrected_and_normalized_entity="Keanu Reeves"),
        ]),
        F(min_maturity_rank=4, max_maturity_rank=4, genres=[Genre.SCI_FI]),
        "HITS: Mad Max: Fury Road (WB + Hardy, R, Sci-Fi), "
        "The Matrix (WB + Reeves, R, Sci-Fi). Each matches 2 entities.",
    ),

    # ── 20 ─────────────────────────────────────────────────────────────────
    (
        "Blumhouse horror, exclude Jordan Peele, PG-13 or under",
        Ent(entity_candidates=[
            E(candidate_entity_phrase="Blumhouse",
              most_likely_category=EntityCategory.STUDIO,
              exclude_from_results=False,
              corrected_and_normalized_entity="Blumhouse Productions"),
            E(candidate_entity_phrase="Jordan Peele",
              most_likely_category=EntityCategory.PERSON,
              exclude_from_results=True,
              corrected_and_normalized_entity="Jordan Peele"),
        ]),
        F(max_maturity_rank=3, genres=[Genre.HORROR]),
        "HITS: Insidious (Blumhouse, Horror, PG-13). "
        "EXCLUDED: Get Out (Peele + R), Saw/Shining/Terrifier (maturity).",
    ),
]

In [ ]:
from dataclasses import asdict
from datetime import datetime
import time

test_cases = LEXICAL_TEST_CASES
for test_case in test_cases:
    print("\n=== TEST CASE ===")
    print(f"{test_case[0]}")
    print(f"Entities: \n   {"\n   ".join([f"{"EXCLUDE" if entity.exclude_from_results else "INCLUDE"} {entity.corrected_and_normalized_entity} ({entity.most_likely_category.upper()})" for entity in test_case[1].entity_candidates])}")
    print(f"Filters:")
    filters = test_case[2]
    if filters:
        if filters.min_release_ts:
            print(f"   min_release: {datetime.fromtimestamp(filters.min_release_ts)}")
        if filters.max_release_ts:
            print(f"   max_release: {datetime.fromtimestamp(filters.max_release_ts)}")
        if filters.min_runtime:
            print(f"   min_runtime: {filters.min_runtime} minutes")
        if filters.max_runtime:
            print(f"   max_runtime: {filters.max_runtime} minutes")
        if filters.min_maturity_rank:
            print(f"   min_maturity_rank: {filters.min_maturity_rank}")
        if filters.max_maturity_rank:
            print(f"   max_maturity_rank: {filters.max_maturity_rank}")
        if filters.genres:
            print(f"   genres: {", ".join([genre.value for genre in filters.genres])}")
        if filters.watch_offer_keys:
            print(f"   watch_providers: {", ".join(filters.watch_offer_keys)}")
    else:
        print("   None")
    print("Expected Results:")
    print(test_case[3])

    start = time.perf_counter()
    search_results = await lexical_search(
        entities=ExtractedEntitiesResponse(entity_candidates=test_case[1].entity_candidates),
        filters=filters,
    )
    elapsed = time.perf_counter() - start

    print("\n=== Search Results ===")
    print(f"{len(search_results)} result(s) in {elapsed:.3f}s")
    for rank, result in enumerate(search_results):
        movie = [movie for movie in movies if movie.tmdb_id == result.movie_id][0]
        print(f"{rank+1}. {movie.title}")
        print(json.dumps(asdict(result), indent=4))


=== TEST CASE ===
Nolan sci-fi without Tom Hardy, post-2010
Entities: 
   INCLUDE Christopher Nolan (PERSON)
   EXCLUDE Tom Hardy (PERSON)
Filters:
   min_release: 2009-12-31 19:00:00
   genres: Sci-Fi
Expected Results:
HITS: Interstellar (Nolan, 2014, Sci-Fi, no Hardy). EXCLUDED: Inception (Hardy stars in it), Dark Knight (not Sci-Fi).

=== Search Results ===
1 result(s) in 0.022s
1. interstellar
{
    "movie_id": 157336,
    "matched_people_count": 1,
    "matched_character_count": 0,
    "matched_studio_count": 0,
    "title_score_sum": 0.0,
    "franchise_score_sum": 0.0,
    "normalized_lexical_score": 1.0
}

=== TEST CASE ===
Spider-Man franchise + Miles Morales, animated PG
Entities: 
   INCLUDE Spider-Man (FRANCHISE)
   INCLUDE Miles Morales (CHARACTER)
Filters:
   min_maturity_rank: 2
   max_maturity_rank: 2
   genres: Animation
Expected Results:
HITS: Spider-Man: Across the Spider-Verse (title tokens 'spider-man', character match, PG, Animation).

=== Search Results ===
1 re

In [14]:
entities = [E(candidate_entity_phrase="of",
              most_likely_category=EntityCategory.MOVIE_TITLE,
              exclude_from_results=True,
              corrected_and_normalized_entity="of"),
              E(candidate_entity_phrase="the",
              most_likely_category=EntityCategory.MOVIE_TITLE,
              exclude_from_results=False,
              corrected_and_normalized_entity="the"),]

start = time.perf_counter()
search_results = await lexical_search(
    entities=ExtractedEntitiesResponse(entity_candidates=entities),
    filters=None,
)
elapsed = time.perf_counter() - start

print("\n=== Search Results ===")
print(f"{len(search_results)} result(s) in {elapsed:.3f}s")
for rank, result in enumerate(search_results):
    movie = [movie for movie in movies if movie.tmdb_id == result.movie_id][0]
    print(f"{rank+1}. {movie.title}")
    print(json.dumps(asdict(result), indent=4))


=== Search Results ===
12 result(s) in 0.078s
1. the pianist
{
    "movie_id": 423,
    "matched_people_count": 0,
    "matched_character_count": 0,
    "matched_studio_count": 0,
    "title_score_sum": 0.8333333333333334,
    "franchise_score_sum": 0.0,
    "normalized_lexical_score": 0.8333333333333334
}
2. the shining
{
    "movie_id": 694,
    "matched_people_count": 0,
    "matched_character_count": 0,
    "matched_studio_count": 0,
    "title_score_sum": 0.8333333333333334,
    "franchise_score_sum": 0.0,
    "normalized_lexical_score": 0.8333333333333334
}
3. the matrix
{
    "movie_id": 603,
    "matched_people_count": 0,
    "matched_character_count": 0,
    "matched_studio_count": 0,
    "title_score_sum": 0.8333333333333334,
    "franchise_score_sum": 0.0,
    "normalized_lexical_score": 0.8333333333333334
}
4. the notebook
{
    "movie_id": 11036,
    "matched_people_count": 0,
    "matched_character_count": 0,
    "matched_studio_count": 0,
    "title_score_sum": 0.833333